<a href="https://colab.research.google.com/github/pradyot-09/DL-Reproducibility-Project/blob/master/distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [0]:
# Import pytorch basic functions/classes
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Import torchvision functions/classes for MNIST import and data loaders
import torchvision
import torchvision.transforms as transforms

# Set device on which code is run
device = 'cuda'

Defining support functions

In [0]:
# Define support function used to convert label to one-hot encoded tensor
def convert_labels(labels):
    target = torch.zeros([len(labels), 10], dtype=torch.float32)
    for i, l in enumerate(labels):
      target[i][l] = 1.0
    return target

Define our network model (the hidden layers size is specified through the constructor)

In [0]:
# Define MLP model and its layers
class Model(nn.Module):

    def __init__(self, hidden_size=1200, dropout=0.0, hidden_dropout=0.0):
        super(Model, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden1 = nn.Linear(784, hidden_size, bias=True)
        self.hidden1_dropout = nn.Dropout(hidden_dropout)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.hidden2_dropout = nn.Dropout(hidden_dropout)
        self.hidden3 = nn.Linear(hidden_size, 10, bias=True)

    def forward(self, x):

        x = self.dropout(x)
        x = F.relu(self.hidden1(x))
        x = self.hidden1_dropout(x)
        x = F.relu(self.hidden2(x))
        x = self.hidden2_dropout(x)
        x = self.hidden3(x)
        return x#, F.softmax(x)

Downloading MNIST dataset

In [0]:
# Define transform from PIL image to tensor and normalize to 1x768 pixels
transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

# Set batch size for data loaders
batch_size = 128

# (Down)load training set
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# (Down)load test set
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Training the Deep Teacher Neural Network

In [61]:
# Setup model and move it to the GPU
net = Model(dropout=0.2, hidden_dropout=0.5)
net.to(device)

# Set up loss function and optimizer: 
#     using cross entropy loss because it's better for classification task

learning_rate = 0.011
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr= learning_rate, momentum=0.9)
#optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.00001)

# Run over 100 epochs (1 epoch = visited all items in dataset)
for epoch in range(100):

    running_loss = 0.0
    total = 0

    if(epoch%20 == 0):

      learning_rate = learning_rate - (0.001) # or maybe decrease by (learning_rate * 0.1)
      optimizer = optim.SGD(net.parameters(), lr= learning_rate, momentum=0.9)

    for i, data in enumerate(trainloader, 0):
        

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)

        # This for not cross entropy
        #target = convert_labels(labels).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        target = labels.to(device).long()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        total += len(data)

        # print statistics
        running_loss += loss.item()
    # print every epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / total))

print('Finished Training')

# Save model after having finished training
PATH = './mnist_dropout_100_epoch.pth'
torch.save(net.state_dict(), PATH)

print('Saved Model')

[1] loss: 0.316
[2] loss: 0.157
[3] loss: 0.124
[4] loss: 0.105
[5] loss: 0.094
[6] loss: 0.086
[7] loss: 0.080
[8] loss: 0.074
[9] loss: 0.070
[10] loss: 0.066
[11] loss: 0.063
[12] loss: 0.059
[13] loss: 0.057
[14] loss: 0.054
[15] loss: 0.053
[16] loss: 0.050
[17] loss: 0.049
[18] loss: 0.047
[19] loss: 0.046
[20] loss: 0.045
[21] loss: 0.042
[22] loss: 0.040
[23] loss: 0.041
[24] loss: 0.040
[25] loss: 0.038
[26] loss: 0.038
[27] loss: 0.037
[28] loss: 0.036
[29] loss: 0.036
[30] loss: 0.034
[31] loss: 0.034
[32] loss: 0.033
[33] loss: 0.033
[34] loss: 0.032
[35] loss: 0.031
[36] loss: 0.031
[37] loss: 0.030
[38] loss: 0.029
[39] loss: 0.029
[40] loss: 0.028
[41] loss: 0.028
[42] loss: 0.026
[43] loss: 0.026
[44] loss: 0.025
[45] loss: 0.025
[46] loss: 0.026
[47] loss: 0.025
[48] loss: 0.025
[49] loss: 0.024
[50] loss: 0.024
[51] loss: 0.024
[52] loss: 0.024
[53] loss: 0.023
[54] loss: 0.023
[55] loss: 0.022
[56] loss: 0.022
[57] loss: 0.023
[58] loss: 0.021
[59] loss: 0.022
[60] l

Run Deep Teacher model on test set

In [62]:
# Instantiate model and load saved network parameters
net = Model().to(device)
net.load_state_dict(torch.load(PATH))

# Run model on test set and determine accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = convert_labels(labels).to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, target = torch.max(target.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

# Output model accuracy to user
print('Accuracy of the network on the 10000 test images: %d %% (%d wrong out of %d)' % (
    100 * correct / total, total - correct, total))

Accuracy of the network on the 10000 test images: 98 % (129 wrong out of 10000)


Train student model to mimic the teacher

In [63]:
import numpy as np
# Custom student loss: linear combination of 2 cross-entropy losses
#     The first one between student output and hard labels
#     The second one between student output and soft labels from teacher

def student_loss(outputA, targetA, outputB, targetB, weight):

    #loss = torch.mean((weight*(outputA - targetA)**2) + (1-weight)*(outputB-targetB)**2)

    loss = weight*(F.cross_entropy(outputA, targetA)) + (1-weight)*(F.cross_entropy(outputB, targetB))

    return loss

# Setup student model and move it to the GPU
student_net = Model(hidden_size = 800)
student_net.to(device)

# Set up loss function and optimizer

optimizer = optim.SGD(student_net.parameters(), lr=0.001, momentum=0.9)
#optimizer = torch.optim.Adam(student_net.parameters(), lr=0.0001)

# Run over 100 epochs (1 epoch = visited all items in dataset)
for epoch in range(100):
    running_loss = 0.0
    total = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = labels.to(device).long() #convert_labels(labels).to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # Set temperature and the weights for losses linear combination
        w = 0.3
        T = 20

        # Compute soft labels using deep teacher model previously trained
        outputs_teacher = net(inputs)
        soft_labels = F.softmax(outputs_teacher/T, dim = 1)

        # Abomination to obtain hard_labels for custom cross entropy loss
        teacher_hard_labels = torch.from_numpy(np.array([np.argmax(l.cpu().detach().numpy()) for l in soft_labels])).to(device).long()

        # Student forward + backward + optimize
        outputs_stud = student_net(inputs)
        loss = student_loss(outputs_stud, target, outputs_stud, teacher_hard_labels, w)
        loss.backward()
        optimizer.step()

        total += len(data)

        # print statistics
        running_loss += loss.item()
    # print every epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / total))

print('Finished Training')

# Save model after having finished training
STUD_PATH = './mnist_student_100_epoch.pth'
torch.save(student_net.state_dict(), STUD_PATH)

print('Saved Model')

[1] loss: 0.640
[2] loss: 0.227
[3] loss: 0.181
[4] loss: 0.163
[5] loss: 0.152
[6] loss: 0.143
[7] loss: 0.135
[8] loss: 0.129
[9] loss: 0.123
[10] loss: 0.116
[11] loss: 0.111
[12] loss: 0.105
[13] loss: 0.101
[14] loss: 0.095
[15] loss: 0.091
[16] loss: 0.087
[17] loss: 0.083
[18] loss: 0.079
[19] loss: 0.076
[20] loss: 0.073
[21] loss: 0.070
[22] loss: 0.067
[23] loss: 0.064
[24] loss: 0.062
[25] loss: 0.059
[26] loss: 0.057
[27] loss: 0.055
[28] loss: 0.053
[29] loss: 0.051
[30] loss: 0.050
[31] loss: 0.048
[32] loss: 0.046
[33] loss: 0.045
[34] loss: 0.043
[35] loss: 0.042
[36] loss: 0.041
[37] loss: 0.039
[38] loss: 0.038
[39] loss: 0.037
[40] loss: 0.036
[41] loss: 0.035
[42] loss: 0.034
[43] loss: 0.033
[44] loss: 0.032
[45] loss: 0.031
[46] loss: 0.030
[47] loss: 0.029
[48] loss: 0.029
[49] loss: 0.028
[50] loss: 0.027
[51] loss: 0.026
[52] loss: 0.026
[53] loss: 0.025
[54] loss: 0.024
[55] loss: 0.024
[56] loss: 0.023
[57] loss: 0.022
[58] loss: 0.022
[59] loss: 0.021
[60] l

Running student model on test set

In [64]:
stud_net = Model(hidden_size = 800).to(device)
stud_net.load_state_dict(torch.load(STUD_PATH))

# Run model on test set and determine accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = convert_labels(labels).to(device)
        outputs = stud_net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, target = torch.max(target.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

# Output model accuracy to user
print('Accuracy of the network on the 10000 test images: %d %% (%d wrong out of %d)' % (
    100 * correct / total, total - correct, total))

Accuracy of the network on the 10000 test images: 97 % (203 wrong out of 10000)
